In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [2]:
labels_to_ids = sql.get_labels_to_ids_dict()

In [3]:
tier1 = '/home/troy/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
tier2 = '/home/troy/Desktop/NDA/complete_nda_tier2_production_ref.json'
nda_names = list(json.load(open(tier1)).keys()) + list(json.load(open(tier2)).keys())
tier1_data = json.load(open(tier1))
tier2_data = json.load(open(tier2))

In [9]:
list(tier2_data)[0]


'12-Item Grit Scale'

In [11]:
tier2_data[list(tier2_data)[0]][0]

{'annotations': [{'tid': 15077,
   'annotation_tid': 15074,
   'value': 'condition : None'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'filterElement : None'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'id : 2081167'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'notes : None'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'required : Required'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'size : None'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'type : GUID'},
  {'tid': 15077, 'annotation_tid': 15074, 'value': 'valueRange : NDAR*'}],
 'label': 'subjectkey (12-Item Grit Scale)',
 'definition': 'The NDAR Global Unique Identifier (GUID) for research subject',
 'id': 15077}

In [11]:
def get_id(pad):
    for value in pad['annotations']:
        if 'id : ' in value.get('value'):
            return value.get('value').split(':')[-1].strip()

data = []
iri = 'http://uri.interlex.org/NDA/uris/datadictionary/elements/'
prefix = 'NDA.CDE:'
for x in tier2_data.items():
    for y in x[1]:
        tid = get_id(y)
        #print(x[0], y['label'], y['id'])
        data.append({'id':y['id'], 'existing_ids':{'iri':iri+tid, 'curie':prefix+tid, 'change':False, 'delete':False}})
data[0]

[{'id': 15179,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/elements/2062112',
   'curie': 'NDA.CDE:2062112',
   'change': False,
   'delete': False}},
 {'id': 15180,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/elements/2062113',
   'curie': 'NDA.CDE:2062113',
   'change': False,
   'delete': False}},
 {'id': 15181,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/elements/2062114',
   'curie': 'NDA.CDE:2062114',
   'change': False,
   'delete': False}},
 {'id': 15182,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/elements/2062115',
   'curie': 'NDA.CDE:2062115',
   'change': False,
   'delete': False}},
 {'id': 15183,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/elements/2062116',
   'curie': 'NDA.CDE:2062116',
   'change': False,
   'delete': False}},
 {'id': 15184,
  'existing_ids': {'iri': 'http://uri.interlex.org/NDA/uris/datadictionary/

In [12]:
json.dump(data, open('/home/troy/Desktop/nda-existing-ids-update.json', 'w'), indent=4)
#python3 /home/troy/Desktop/interlexutils/interlexutils/api_wrapper.py updateTerms -f /home/troy/Desktop/nda-existing-ids-update.json -p

In [25]:
annotations=sql.get_annotations()

In [26]:
terms = sql.get_terms()

In [17]:
exids = sql.get_existing_ids()

In [18]:
exids.loc[exids.iri.str.contains('http://uri.interlex.org/NDA/uris/datadictionary/elements/')]

,id,tid,curie,iri,preferred,ilx,type,label,definition
234877,15090,15090,NDA.CDE:2081180,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115087,cde,grit9 (12-Item Grit Scale),I finish whatever I begin.
234878,15084,15084,NDA.CDE:2081174,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115081,cde,grit3 (12-Item Grit Scale),My interests change from year to year.
234879,15079,15079,NDA.CDE:2081169,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115076,cde,interview_date (12-Item Grit Scale),Date on which the interview/genetic test/sampl...
234882,15086,15086,NDA.CDE:2081176,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115083,cde,grit5 (12-Item Grit Scale),I have been obsessed with a certain idea or pr...
234891,15100,15100,NDA.CDE:2252688,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115097,cde,grit_amb2 (12-Item Grit Scale),I am ambitious
234892,15087,15087,NDA.CDE:2081177,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115084,cde,grit6 (12-Item Grit Scale),I am a hard worker.
234897,15085,15085,NDA.CDE:2081175,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115082,cde,grit4 (12-Item Grit Scale),Setbacks don&#39;t discourage me.
234898,15089,15089,NDA.CDE:2081179,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115086,cde,grit8 (12-Item Grit Scale),I have difficulty maintaining my focus on proj...
234899,15102,15102,NDA.CDE:2252690,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115099,cde,grit_amb4 (12-Item Grit Scale),I think achievement is overrated
234900,15101,15101,NDA.CDE:2252689,http://uri.interlex.org/NDA/uris/datadictionar...,0,ilx_0115098,cde,grit_amb3 (12-Item Grit Scale),Achieving something of lasting importance is t...
